In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import io
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pdb

BATCH_SIZE = 32
VOCAB_SIZE = 50000

### Helper functions for each step in the pipeline

In [20]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    print(n, d)
    i = 0
    for line in fin:
        if i == VOCAB_SIZE:
            break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
        i += 1
    return data

In [21]:
from collections import Counter
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = 50000

def build_vocab():
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    word_vectors = pkl.load(open("fasttext_word_vectors.p", "rb"))
    id2token = list(word_vectors.keys())
    token2id = dict(zip(word_vectors, range(2,2+len(word_vectors)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return word_vectors, token2id, id2token

In [22]:
def convert_labels_to_integers(data_label):
    for i in range(len(data_label)):
        if data_label[i] == "contradiction":
            data_label[i] = 0
        elif data_label[i] == "entailment":
            data_label[i] = 1
        elif data_label[i] == "neutral":
            data_label[i] = 2
    return data_label

In [23]:
def verify_order(sent1_data, sent2_data, data_label):
    i = random.randint(1, len(sent1_data))
    print(sent1_data[i])
    print(sent2_data[i])
    print(data_label[i])

In [24]:
# Word tokenize each entry in a list of sentences
def tokenize(sentence_list):
    return [word_tokenize(sentence_list[i]) for i in range(len(sentence_list))]

In [25]:
# "one-hot encode": convert each token to id in vocabulary vector (token2id)
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data 

### Creating vocabulary & embedding matrix from FastText vectors

In [26]:
word_vectors, token2id, id2token = build_vocab()

In [27]:
_weights = np.array(list(word_vectors.values()))
pad_vec = np.zeros((1, 300))
unk_vec = np.random.randn(1, 300) * 0.01
pad_unk_vecs = np.vstack((pad_vec, unk_vec))
_WEIGHTS = np.vstack((pad_unk_vecs, _weights))
_WEIGHTS.shape

(50002, 300)

### Function to pre-process data for TwoSentenceModel
#### Shuffle, word tokenize, one-hot index into vocabulary

In [28]:
def text_pipeline(sent1s, sent2s, labels, verify=True):
    labels = convert_labels_to_integers(labels)
    seed = random.randint(1, 100)
    print("Random seed for shuffling: {}".format(seed))
    random.Random(seed).shuffle(sent1s)
    random.Random(seed).shuffle(sent2s)
    random.Random(seed).shuffle(labels)
    
    print("\nVerifying that the data and label match after shuffling")
    if verify:
        verify_order(sent1s, sent2s, labels)
        verify_order(sent1s, sent2s, labels)
          
    print("\nTokenizing sentence 1 list...")    
    sent1s_tokenized = tokenize(sent1s)
    print("done!")
    print("\nTokenizing sentence 2 list... ")  
    sent2s_tokenized = tokenize(sent2s)
    print("done!")
    
    print("\nOne-hot encoding words for sentence 1 list...")  
    sent1s_indices = token2index_dataset(sent1s_tokenized)
    print("done!")
    print("\nOne-hot encoding words for sentence 2 list...")  
    sent2s_indices = token2index_dataset(sent2s_tokenized)
    print("done!")
    
    return (sent1s_indices, sent2s_indices, labels)

### DataLoader

In [29]:
# MAX_SENTENCE_LENGTH = 30

import numpy as np
import torch
from torch.utils.data import Dataset

class TwoSentencesDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    """
    
    def __init__(self, sent1_data_list, sent2_data_list, target_list):
        """
        @param sent1_data_list: list of sentence1's (index matches sentence2's and target_list below)
        @param sent2_data_list: list of sentence2's
        @param target_list: list of correct labels

        """
        self.sent1_data_list = sent1_data_list
        self.sent2_data_list = sent2_data_list
        self.target_list = target_list
        assert (len(self.sent1_data_list) == len(self.target_list) and len(self.sent2_data_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_data_list)
        
    def __getitem__(self, key):
        ###
        ### Returns [[sentence, 1, tokens], [sentence, 2, tokens]]
        ###
        """
        Triggered when you call dataset[i]
        """
        sent1_tokens_idx = self.sent1_data_list[key][:MAX_SENTENCE_LENGTH]
        sent2_tokens_idx = self.sent2_data_list[key][:MAX_SENTENCE_LENGTH]
        combined_tokens_idx = [sent1_tokens_idx, sent2_tokens_idx]
        label = self.target_list[key]
        return [combined_tokens_idx, len(sent1_tokens_idx), len(sent2_tokens_idx), label]

def twosentences_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    sent1_data_list = []
    sent2_data_list = []
    sent1_length_list = []
    sent2_length_list = []
    label_list = []
    combined_data_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[3])
        sent1_length_list.append(datum[1])
        sent2_length_list.append(datum[2])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        combined_data_list.append([padded_vec_1, padded_vec_2])
    return [torch.from_numpy(np.array(combined_data_list)), 
            torch.LongTensor(sent1_length_list), torch.LongTensor(sent2_length_list), torch.LongTensor(label_list)]

### Train Dataset creation

In [30]:
snli_train = pd.read_csv('snli_train.tsv', sep='\t')
TRAIN_SIZE = 100000

sent1_data = list(snli_train["sentence1"])[:TRAIN_SIZE]
sent2_data = list(snli_train["sentence2"])[:TRAIN_SIZE]
data_label = list(snli_train["label"])[:TRAIN_SIZE]
print("Size of training data: {}".format(len(sent1_data)))

Size of training data: 100000


In [31]:
sent1_train_indices, sent2_train_indices, train_label = text_pipeline(sent1_data, sent2_data, data_label)
train_dataset = TwoSentencesDataset(sent1_train_indices, sent2_train_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )
print("Finished creating train_loader.")

Random seed for shuffling: 37

Verifying that the data and label match after shuffling
A small boy wearing a red hat is standing in a street .
A little boy is outside .
1
An elderly women buying shampoo and conditioner at a store .
The woman is buying a computer .
0

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!
Finished creating train_loader.


### Val Dataset creation

In [32]:
snli_val = pd.read_csv('snli_val.tsv', sep='\t')
sent1_val = list(snli_val["sentence1"])
sent2_val = list(snli_val["sentence2"])
val_label = list(snli_val["label"])
print("Size of val data: {}".format(len(sent1_val)))

Size of val data: 1000


In [33]:
sent1_val_indices, sent2_val_indices, val_label = text_pipeline(sent1_val, sent2_val, val_label)
val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

Random seed for shuffling: 82

Verifying that the data and label match after shuffling
A skateboarder goes down a railing .
A skateboarder goes up the hill .
0
A man and a woman hug on a grassy hillside overlooking the countryside in the distance .
The man and woman are at a skating rink .
0

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!


In [34]:
pd.Series([len(x) for x in snli_train['sentence1']]).describe()['75%']
MAX_SENTENCE_LENGTH = 40

### CNN Model

In [35]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(_WEIGHTS))
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool1d(30)
        self.linear1 = nn.Linear(2*hidden_size, 100)
        self.linear2 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, sent1_lengths, sent2_lengths):
        
        batch_size = x.size()[0]
        seq_len = x.size()[2]
        
        sent1s = torch.tensor(x[:, 0, :]).cuda()
        sent2s = torch.tensor(x[:, 1, :]).cuda()
        ordered_sents = torch.cat([sent1s, sent2s], dim=0).cuda()

        embed = self.embedding(ordered_sents)
        hidden = self.conv1(embed.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))

        hidden = self.conv2(hidden.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(2*batch_size, seq_len, hidden.size(-1))
        hidden = self.maxpool(hidden.transpose(1, 2)).transpose(1, 2).squeeze(dim=1)
        
        hidden_sent1s = hidden[0:batch_size, :]
        hidden_sent2s = hidden[batch_size:, :]     
        
        linear1 = self.linear1(torch.cat([hidden_sent1s, hidden_sent2s], dim=1))
        linear1 = F.relu(linear1.contiguous().view(-1, linear1.size(-1))).view(linear1.shape)
        logits = self.linear2(linear1)
        return logits

In [39]:
# Function for testing the model
def test_model(loader, model):
    """
    Helper function that tests the model's performance on a dataset
    """
    correct = 0
    total = 0
    model.eval()
    for (data, sent1_lengths, sent2_lengths, labels) in loader:
        data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
        outputs = F.softmax(model(data_batch, sent1_length_batch, sent2_length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        labels = labels.cuda()
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def train_model(model, lr = 0.001, num_epochs = 7, criterion = nn.CrossEntropyLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
    max_val_acc = 0
    for epoch in range(num_epochs):
        for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.cuda(), sent1_lengths.cuda(), sent2_lengths.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(data_batch, sent1_length_batch, sent2_length_batch)
            loss = criterion(outputs, label_batch)
            
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                if val_acc > max_val_acc:
                    max_val_acc = val_acc
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc))
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), loss))
                
    print("Max Validation Accuracy: {}".format(max_val_acc))
    return max_val_acc

In [38]:
model = CNN(emb_size = 300, hidden_size=300, num_layers=1, num_classes=3).cuda()
train_model(model)

Epoch: [1/7], Step: [101/3125], Validation Acc: 41.7
Epoch: [1/7], Step: [101/3125], Training Loss: 1.1114627122879028
Epoch: [1/7], Step: [201/3125], Validation Acc: 44.8
Epoch: [1/7], Step: [201/3125], Training Loss: 0.9520798921585083
Epoch: [1/7], Step: [301/3125], Validation Acc: 47.4
Epoch: [1/7], Step: [301/3125], Training Loss: 1.023442268371582
Epoch: [1/7], Step: [401/3125], Validation Acc: 48.7
Epoch: [1/7], Step: [401/3125], Training Loss: 0.9866016507148743
Epoch: [1/7], Step: [501/3125], Validation Acc: 50.5
Epoch: [1/7], Step: [501/3125], Training Loss: 1.1257799863815308
Epoch: [1/7], Step: [601/3125], Validation Acc: 51.8
Epoch: [1/7], Step: [601/3125], Training Loss: 0.9397417306900024
Epoch: [1/7], Step: [701/3125], Validation Acc: 51.9
Epoch: [1/7], Step: [701/3125], Training Loss: 1.0343130826950073
Epoch: [1/7], Step: [801/3125], Validation Acc: 55.1
Epoch: [1/7], Step: [801/3125], Training Loss: 0.9194968342781067
Epoch: [1/7], Step: [901/3125], Validation Acc: 5

Epoch: [3/7], Step: [801/3125], Validation Acc: 65.2
Epoch: [3/7], Step: [801/3125], Training Loss: 0.6371939182281494
Epoch: [3/7], Step: [901/3125], Validation Acc: 65.1
Epoch: [3/7], Step: [901/3125], Training Loss: 0.8485713601112366
Epoch: [3/7], Step: [1001/3125], Validation Acc: 64.2
Epoch: [3/7], Step: [1001/3125], Training Loss: 0.7629712820053101
Epoch: [3/7], Step: [1101/3125], Validation Acc: 63.1
Epoch: [3/7], Step: [1101/3125], Training Loss: 0.7933327555656433
Epoch: [3/7], Step: [1201/3125], Validation Acc: 63.0
Epoch: [3/7], Step: [1201/3125], Training Loss: 0.7301148176193237
Epoch: [3/7], Step: [1301/3125], Validation Acc: 64.1
Epoch: [3/7], Step: [1301/3125], Training Loss: 0.5719056725502014
Epoch: [3/7], Step: [1401/3125], Validation Acc: 63.0
Epoch: [3/7], Step: [1401/3125], Training Loss: 0.6100853681564331
Epoch: [3/7], Step: [1501/3125], Validation Acc: 64.7
Epoch: [3/7], Step: [1501/3125], Training Loss: 0.7864716649055481
Epoch: [3/7], Step: [1601/3125], Val

Epoch: [5/7], Step: [1501/3125], Validation Acc: 65.4
Epoch: [5/7], Step: [1501/3125], Training Loss: 0.7293466329574585
Epoch: [5/7], Step: [1601/3125], Validation Acc: 65.3
Epoch: [5/7], Step: [1601/3125], Training Loss: 0.7243886590003967
Epoch: [5/7], Step: [1701/3125], Validation Acc: 65.4
Epoch: [5/7], Step: [1701/3125], Training Loss: 0.5606755614280701
Epoch: [5/7], Step: [1801/3125], Validation Acc: 65.4
Epoch: [5/7], Step: [1801/3125], Training Loss: 0.6346427202224731
Epoch: [5/7], Step: [1901/3125], Validation Acc: 64.3
Epoch: [5/7], Step: [1901/3125], Training Loss: 0.7993612885475159
Epoch: [5/7], Step: [2001/3125], Validation Acc: 65.4
Epoch: [5/7], Step: [2001/3125], Training Loss: 0.46721622347831726
Epoch: [5/7], Step: [2101/3125], Validation Acc: 66.4
Epoch: [5/7], Step: [2101/3125], Training Loss: 0.8610687255859375
Epoch: [5/7], Step: [2201/3125], Validation Acc: 66.2
Epoch: [5/7], Step: [2201/3125], Training Loss: 0.529364287853241
Epoch: [5/7], Step: [2301/3125],

Epoch: [7/7], Step: [2101/3125], Validation Acc: 65.8
Epoch: [7/7], Step: [2101/3125], Training Loss: 0.8339399695396423
Epoch: [7/7], Step: [2201/3125], Validation Acc: 66.0
Epoch: [7/7], Step: [2201/3125], Training Loss: 0.44701504707336426
Epoch: [7/7], Step: [2301/3125], Validation Acc: 66.0
Epoch: [7/7], Step: [2301/3125], Training Loss: 0.4254485070705414
Epoch: [7/7], Step: [2401/3125], Validation Acc: 66.0
Epoch: [7/7], Step: [2401/3125], Training Loss: 0.7201557159423828
Epoch: [7/7], Step: [2501/3125], Validation Acc: 65.1
Epoch: [7/7], Step: [2501/3125], Training Loss: 0.5972468256950378
Epoch: [7/7], Step: [2601/3125], Validation Acc: 66.5
Epoch: [7/7], Step: [2601/3125], Training Loss: 0.46032217144966125
Epoch: [7/7], Step: [2701/3125], Validation Acc: 65.5
Epoch: [7/7], Step: [2701/3125], Training Loss: 0.4536064863204956
Epoch: [7/7], Step: [2801/3125], Validation Acc: 66.1
Epoch: [7/7], Step: [2801/3125], Training Loss: 0.6918835639953613
Epoch: [7/7], Step: [2901/3125

### Evaluating on MNLI

In [6]:
mnli_val = pd.read_csv('mnli_val.tsv', sep='\t')

In [8]:
sentence_length_75 = pd.Series([len(x) for x in mnli_val['sentence1']]).describe()['75%']
print("75th percentile for sentence length (in characters): {}".format(sentence_length_75))
MAX_SENTENCE_LENGTH = 40

75th percentile for sentence length (in characters): 151.0


#### Build dictionary of (sent1, sent2, label) data, by genre.

In [12]:
mnli_val_dict = {}
for x in mnli_val['genre'].unique():
    filtered = mnli_val[mnli_val['genre'] == x]
    mnli_val_dict[x] = {}
    mnli_val_dict[x]["sent1s"] = list(filtered["sentence1"])
    mnli_val_dict[x]["sent2s"] = list(filtered["sentence2"])
    mnli_val_dict[x]["label"] = convert_labels_to_integers(list(filtered["label"]))

In [42]:
for x in mnli_val_dict.keys():
    print(len(mnli_val_dict[x]["sent1s"]))

995
1005
1002
1016
982


In [16]:
# quick verify
verify_order(mnli_val_dict['travel']["sent1s"], mnli_val_dict['travel']["sent2s"], mnli_val_dict['travel']["label"])

To the south , the former fishing villages of Sorrento and Positano spill down the craggy cliffs of the serpentine Amalfi coast , justifiably tauted as one of the world 's most beautiful drives .
Sorrento used to be a fishing village .
1


In [41]:
# for each genre, build validation set and evaluate on it. 
cnn_results = {}
for genre in mnli_val_dict.keys():
    sent1_val_indices, sent2_val_indices, val_label = text_pipeline(mnli_val_dict[genre]["sent1s"], 
                                                                    mnli_val_dict[genre]["sent2s"], 
                                                                    mnli_val_dict[genre]["label"])
    val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                             batch_size=BATCH_SIZE, 
                                             collate_fn=twosentences_collate_func,
                                             #shuffle=True
                                             )
    cnn_results[genre] = test_model(val_loader, model)
    print("Genre {} has validation accuracy: {}".format(genre, cnn_results[genre]))
    

Random seed for shuffling: 51

Verifying that the data and label match after shuffling
Good sir , Jon began .
Jon addressed the man .
1
The day my deadline came , I got a business card .
The deadline to accept my promotion arrived and I got a business card with my new title .
2

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!
Genre fiction has validation accuracy: 46.130653266331656
Random seed for shuffling: 29

Verifying that the data and label match after shuffling
really oh i thought it was great yeah
That was horrible
0
i think there would be an awful lot of resentment and um i i really do n't think it would be feasible on our country
The war would lead to a bunch of resentment among civilians .
2

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for 

### MNLI results

| genre      | CNN (val accuracy) | RNN (val accuracy) |
|------------|--------------------|--------------------|
| fiction    |       46.13%       |                    |
| telephone  |       45.97%       |                    |
| travel     |       48.17%       |                    |
| slate      |       44.61%       |                    |
| government |       51.38%       |                    |

In [ ]:
###
### Ideas
###

# Dropout layers --> prob 0.5
# weight decay. 

#rnn --> layer normalize

# CNN masking
# do not backpropagate
# after conv, cresate tensor masked not update.
# right after regular linear layer
# 
# set all elements until padding to 1